In [1]:
import numpy as np


unique_cells = sorted(set("""
r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . P . . .
. . . . . N . .
P P P P . P P P
R N B Q K B . R"""))

unique_cells.remove("\n")
unique_cells.remove(" ")

unique_figures = np.array(list(set([x.upper() for x in unique_cells[1:]])))


fig_dict = {n : i for i, n in enumerate(unique_cells)}
num_dict = {i : n for i, n in enumerate(unique_cells)}


def fig_to_num(f): return fig_dict[f]
def num_to_fig(n): return num_dict[n]


def board_to_input(board):
    inp = []
    board = str(board).replace("\n", '').replace(" ", '')
    board.replace(" ", '')
    for s in board:
        inp.append(fig_to_num(s))
    return inp


def input_to_board(inp):
    board = []
    for n in inp:
        board.append(num_to_fig(n))
    return np.array(board).reshape(8, 8)


board_pos_dict = {p : n for n, p in enumerate("abcdefgh")}
num_pos_dict = {n : p for n, p in enumerate("abcdefgh")}


def board_pos_to_num_pos(b_pos):
    return np.array([8 - int(b_pos[1]), board_pos_dict[b_pos[0]]])

def num_pos_to_board_pos(pos):
    return f"{num_pos_dict[pos[1]]}{8 - pos[0]}"


def move_to_output(board, uci_move):
    output = np.zeros(4)
    from_pos = board_pos_to_num_pos(uci_move[:2])
    to_pos = board_pos_to_num_pos(uci_move[2:])

    # set the pos
    output[:2] = from_pos
    output[2:] = to_pos
    return output


def binary_4bits(x):
    return {
        0 : [0, 0, 0, 0],
        1 : [0, 0, 0, 1],
        2 : [0, 0, 1, 0],
        3 : [0, 0, 1, 1],
        4 : [0, 1, 0, 0],
        5 : [0, 1, 0, 1],
        6 : [0, 1, 1, 0],
        7 : [0, 1, 1, 1],
        8 : [1, 0, 0, 0],
        9 : [1, 0, 0, 1],
        10 : [1, 0, 1, 0],
        11 : [1, 0, 1, 1],
        12 : [1, 1, 0, 0],
    }[x]


def binary_3bits(x):
    return {
        0 : [0, 0, 0],
        1 : [0, 0, 1],
        2 : [0, 1, 0],
        3 : [0, 1, 1],
        4 : [1, 0, 0],
        5 : [1, 0, 1],
        6 : [1, 1, 0],
        7 : [1, 1, 1],
    }[x]


def to_decimal(x):
    bits = [1 if x_ >= 0.5 else 0 for x_ in x]
    if len(bits) == 3:
        for i in range(0, 8):
          if np.all(binary_3bits(i) == bits):
            return i
    elif len(bits) == 4:
        for i in range(0, 13):
          if np.all(binary_4bits(i) == bits):
            return i


if __name__ == "__main__":
    print(to_decimal([1, 1, 0, 0]))
    print(to_decimal([1, 0, 0, 0]))
    print(to_decimal([1, 0, 0]))
    print(to_decimal([1, 1, 1]))
    print(binary_3bits(7))
    print(binary_4bits(8))


12
8
4
7
[1, 1, 1]
[1, 0, 0, 0]


In [2]:
!pip install python-chess==0.31.4

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 134 kB 1.1 MB/s eta 0:00:01
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import chess

import gc
import os
import copy
import json
import pickle

In [5]:
games_json = {}

with open("../dataset/aronian_games.json", "r") as file:
    games_json = json.load(file)
    
bad_games = []

In [6]:
for k, v in games_json.items():
    board = chess.Board()
    try:
        for mk, mv in v["moves"].items():
            if mk == "result": continue
            wm = mv.split()[0]
            board.push_san(wm)
            if len(mv.split()) == 2:
                bm = mv.split()[1]
                board.push_san(bm)
        print(f"Game {k} was sucessfully played.")
    except Exception as ex:
        bad_games.append(k)
        print(ex)

Game 1 was sucessfully played.
Game 2 was sucessfully played.
Game 3 was sucessfully played.
Game 4 was sucessfully played.
Game 5 was sucessfully played.
Game 6 was sucessfully played.
Game 7 was sucessfully played.
Game 8 was sucessfully played.
Game 9 was sucessfully played.
Game 10 was sucessfully played.
Game 11 was sucessfully played.
Game 12 was sucessfully played.
Game 13 was sucessfully played.
Game 14 was sucessfully played.
Game 15 was sucessfully played.
Game 16 was sucessfully played.
Game 17 was sucessfully played.
Game 18 was sucessfully played.
Game 19 was sucessfully played.
Game 20 was sucessfully played.
Game 21 was sucessfully played.
Game 22 was sucessfully played.
Game 23 was sucessfully played.
Game 24 was sucessfully played.
Game 25 was sucessfully played.
Game 26 was sucessfully played.
Game 27 was sucessfully played.
Game 28 was sucessfully played.
Game 29 was sucessfully played.
Game 30 was sucessfully played.
Game 31 was sucessfully played.
Game 32 was suces

Game 273 was sucessfully played.
Game 274 was sucessfully played.
Game 275 was sucessfully played.
Game 276 was sucessfully played.
Game 277 was sucessfully played.
Game 278 was sucessfully played.
Game 279 was sucessfully played.
Game 280 was sucessfully played.
Game 281 was sucessfully played.
Game 282 was sucessfully played.
Game 283 was sucessfully played.
Game 284 was sucessfully played.
Game 285 was sucessfully played.
Game 286 was sucessfully played.
Game 287 was sucessfully played.
Game 288 was sucessfully played.
Game 289 was sucessfully played.
Game 290 was sucessfully played.
Game 291 was sucessfully played.
Game 292 was sucessfully played.
Game 293 was sucessfully played.
Game 294 was sucessfully played.
Game 295 was sucessfully played.
Game 296 was sucessfully played.
Game 297 was sucessfully played.
Game 298 was sucessfully played.
Game 299 was sucessfully played.
Game 300 was sucessfully played.
Game 301 was sucessfully played.
Game 302 was sucessfully played.
Game 303 w

Game 591 was sucessfully played.
Game 592 was sucessfully played.
Game 593 was sucessfully played.
Game 594 was sucessfully played.
Game 595 was sucessfully played.
Game 596 was sucessfully played.
Game 597 was sucessfully played.
Game 598 was sucessfully played.
Game 599 was sucessfully played.
Game 600 was sucessfully played.
Game 601 was sucessfully played.
Game 602 was sucessfully played.
Game 603 was sucessfully played.
Game 604 was sucessfully played.
Game 605 was sucessfully played.
Game 606 was sucessfully played.
Game 607 was sucessfully played.
Game 608 was sucessfully played.
Game 609 was sucessfully played.
Game 610 was sucessfully played.
Game 611 was sucessfully played.
Game 612 was sucessfully played.
Game 613 was sucessfully played.
Game 614 was sucessfully played.
Game 615 was sucessfully played.
Game 616 was sucessfully played.
Game 617 was sucessfully played.
Game 618 was sucessfully played.
Game 619 was sucessfully played.
Game 620 was sucessfully played.
Game 621 w

Game 885 was sucessfully played.
Game 886 was sucessfully played.
Game 887 was sucessfully played.
Game 888 was sucessfully played.
Game 889 was sucessfully played.
Game 890 was sucessfully played.
Game 891 was sucessfully played.
Game 892 was sucessfully played.
Game 893 was sucessfully played.
Game 894 was sucessfully played.
Game 895 was sucessfully played.
Game 896 was sucessfully played.
Game 897 was sucessfully played.
Game 898 was sucessfully played.
Game 899 was sucessfully played.
Game 900 was sucessfully played.
Game 901 was sucessfully played.
Game 902 was sucessfully played.
Game 903 was sucessfully played.
Game 904 was sucessfully played.
Game 905 was sucessfully played.
Game 906 was sucessfully played.
Game 907 was sucessfully played.
Game 908 was sucessfully played.
Game 909 was sucessfully played.
Game 910 was sucessfully played.
Game 911 was sucessfully played.
Game 912 was sucessfully played.
Game 913 was sucessfully played.
Game 914 was sucessfully played.
Game 915 w

Game 1180 was sucessfully played.
Game 1181 was sucessfully played.
Game 1182 was sucessfully played.
Game 1183 was sucessfully played.
Game 1184 was sucessfully played.
Game 1185 was sucessfully played.
Game 1186 was sucessfully played.
Game 1187 was sucessfully played.
Game 1188 was sucessfully played.
Game 1189 was sucessfully played.
Game 1190 was sucessfully played.
Game 1191 was sucessfully played.
Game 1192 was sucessfully played.
Game 1193 was sucessfully played.
Game 1194 was sucessfully played.
Game 1195 was sucessfully played.
Game 1196 was sucessfully played.
Game 1197 was sucessfully played.
Game 1198 was sucessfully played.
Game 1199 was sucessfully played.
Game 1200 was sucessfully played.
Game 1201 was sucessfully played.
Game 1202 was sucessfully played.
Game 1203 was sucessfully played.
Game 1204 was sucessfully played.
Game 1205 was sucessfully played.
Game 1206 was sucessfully played.
Game 1207 was sucessfully played.
Game 1208 was sucessfully played.
Game 1209 was 

...

Game 2082 was sucessfully played.
Game 2083 was sucessfully played.
Game 2084 was sucessfully played.
Game 2085 was sucessfully played.
Game 2086 was sucessfully played.
Game 2087 was sucessfully played.
Game 2088 was sucessfully played.
Game 2089 was sucessfully played.
Game 2090 was sucessfully played.
Game 2091 was sucessfully played.
Game 2092 was sucessfully played.
Game 2093 was sucessfully played.
Game 2094 was sucessfully played.
Game 2095 was sucessfully played.
Game 2096 was sucessfully played.
Game 2097 was sucessfully played.
Game 2098 was sucessfully played.
Game 2099 was sucessfully played.
Game 2100 was sucessfully played.
Game 2101 was sucessfully played.
Game 2102 was sucessfully played.
Game 2103 was sucessfully played.
Game 2104 was sucessfully played.
Game 2105 was sucessfully played.
Game 2106 was sucessfully played.
Game 2107 was sucessfully played.
Game 2108 was sucessfully played.
Game 2109 was sucessfully played.
Game 2110 was sucessfully played.
Game 2111 was 

Game 2366 was sucessfully played.
Game 2367 was sucessfully played.
Game 2368 was sucessfully played.
Game 2369 was sucessfully played.
Game 2370 was sucessfully played.
Game 2371 was sucessfully played.
Game 2372 was sucessfully played.
Game 2373 was sucessfully played.
Game 2374 was sucessfully played.
Game 2375 was sucessfully played.
Game 2376 was sucessfully played.
Game 2377 was sucessfully played.
Game 2378 was sucessfully played.
Game 2379 was sucessfully played.
Game 2380 was sucessfully played.
Game 2381 was sucessfully played.
Game 2382 was sucessfully played.
Game 2383 was sucessfully played.
Game 2384 was sucessfully played.
Game 2385 was sucessfully played.
Game 2386 was sucessfully played.
Game 2387 was sucessfully played.
Game 2388 was sucessfully played.
Game 2389 was sucessfully played.
Game 2390 was sucessfully played.
Game 2391 was sucessfully played.
Game 2392 was sucessfully played.
Game 2393 was sucessfully played.
Game 2394 was sucessfully played.
Game 2395 was 

Game 2674 was sucessfully played.
Game 2675 was sucessfully played.
Game 2676 was sucessfully played.
Game 2677 was sucessfully played.
Game 2678 was sucessfully played.
Game 2679 was sucessfully played.
Game 2680 was sucessfully played.
Game 2681 was sucessfully played.
Game 2682 was sucessfully played.
Game 2683 was sucessfully played.
Game 2684 was sucessfully played.
Game 2685 was sucessfully played.
Game 2686 was sucessfully played.
Game 2687 was sucessfully played.
Game 2688 was sucessfully played.
Game 2689 was sucessfully played.
Game 2690 was sucessfully played.
Game 2691 was sucessfully played.
Game 2692 was sucessfully played.
Game 2693 was sucessfully played.
Game 2694 was sucessfully played.
Game 2695 was sucessfully played.
Game 2696 was sucessfully played.
Game 2697 was sucessfully played.
Game 2698 was sucessfully played.
Game 2699 was sucessfully played.
Game 2700 was sucessfully played.
Game 2701 was sucessfully played.
Game 2702 was sucessfully played.
Game 2703 was 

Game 2983 was sucessfully played.
Game 2984 was sucessfully played.
Game 2985 was sucessfully played.
Game 2986 was sucessfully played.
Game 2987 was sucessfully played.
Game 2988 was sucessfully played.
Game 2989 was sucessfully played.
Game 2990 was sucessfully played.
Game 2991 was sucessfully played.
Game 2992 was sucessfully played.
Game 2993 was sucessfully played.
Game 2994 was sucessfully played.
Game 2995 was sucessfully played.
Game 2996 was sucessfully played.
Game 2997 was sucessfully played.
Game 2998 was sucessfully played.
Game 2999 was sucessfully played.
Game 3000 was sucessfully played.
Game 3001 was sucessfully played.
Game 3002 was sucessfully played.
Game 3003 was sucessfully played.
Game 3004 was sucessfully played.
Game 3005 was sucessfully played.
Game 3006 was sucessfully played.
Game 3007 was sucessfully played.
Game 3008 was sucessfully played.
Game 3009 was sucessfully played.
Game 3010 was sucessfully played.
Game 3011 was sucessfully played.
Game 3012 was 

Game 3297 was sucessfully played.
Game 3298 was sucessfully played.
Game 3299 was sucessfully played.
Game 3300 was sucessfully played.
Game 3301 was sucessfully played.
Game 3302 was sucessfully played.
Game 3303 was sucessfully played.
Game 3304 was sucessfully played.
Game 3305 was sucessfully played.
Game 3306 was sucessfully played.
Game 3307 was sucessfully played.
Game 3308 was sucessfully played.
Game 3309 was sucessfully played.
Game 3310 was sucessfully played.
Game 3311 was sucessfully played.
Game 3312 was sucessfully played.
Game 3313 was sucessfully played.
Game 3314 was sucessfully played.
Game 3315 was sucessfully played.
Game 3316 was sucessfully played.
Game 3317 was sucessfully played.
Game 3318 was sucessfully played.
Game 3319 was sucessfully played.
Game 3320 was sucessfully played.
Game 3321 was sucessfully played.
Game 3322 was sucessfully played.
Game 3323 was sucessfully played.
Game 3324 was sucessfully played.
Game 3325 was sucessfully played.
Game 3326 was 

In [7]:
bad_games

[]

In [8]:
not_conv_games = []

In [9]:
input_data = []
output_data = []

for k, v in games_json.items():
    
    board = chess.Board()
    try:
        for mk, mv in v["moves"].items():
            if mk == "result": continue

            input_data.append(board_to_input(board))
            wm = mv.split()[0]
            board.push_san(wm)

            output_data.append([board_pos_to_num_pos(str(board.move_stack[-1])[:2]), 
                                board_pos_to_num_pos(str(board.move_stack[-1])[2:])])

            if len(mv.split()) == 2:
                bm = mv.split()[1]
                board.push_san(bm)
        print(f"Game {k} is converted.")
    except Exception as ex:
        print(ex)
        not_conv_games.append(k)

Game 1 is converted.
Game 2 is converted.
Game 3 is converted.
Game 4 is converted.
Game 5 is converted.
Game 6 is converted.
Game 7 is converted.
Game 8 is converted.
Game 9 is converted.
Game 10 is converted.
Game 11 is converted.
Game 12 is converted.
Game 13 is converted.
Game 14 is converted.
Game 15 is converted.
Game 16 is converted.
Game 17 is converted.
Game 18 is converted.
Game 19 is converted.
Game 20 is converted.
Game 21 is converted.
Game 22 is converted.
Game 23 is converted.
Game 24 is converted.
Game 25 is converted.
Game 26 is converted.
Game 27 is converted.
Game 28 is converted.
Game 29 is converted.
Game 30 is converted.
Game 31 is converted.
Game 32 is converted.
Game 33 is converted.
Game 34 is converted.
Game 35 is converted.
Game 36 is converted.
Game 37 is converted.
Game 38 is converted.
Game 39 is converted.
Game 40 is converted.
Game 41 is converted.
Game 42 is converted.
Game 43 is converted.
Game 44 is converted.
Game 45 is converted.
Game 46 is converte

Game 367 is converted.
Game 368 is converted.
Game 369 is converted.
Game 370 is converted.
Game 371 is converted.
Game 372 is converted.
Game 373 is converted.
Game 374 is converted.
Game 375 is converted.
Game 376 is converted.
Game 377 is converted.
Game 378 is converted.
Game 379 is converted.
Game 380 is converted.
Game 381 is converted.
Game 382 is converted.
Game 383 is converted.
Game 384 is converted.
Game 385 is converted.
Game 386 is converted.
Game 387 is converted.
Game 388 is converted.
Game 389 is converted.
Game 390 is converted.
Game 391 is converted.
Game 392 is converted.
Game 393 is converted.
Game 394 is converted.
Game 395 is converted.
Game 396 is converted.
Game 397 is converted.
Game 398 is converted.
Game 399 is converted.
Game 400 is converted.
Game 401 is converted.
Game 402 is converted.
Game 403 is converted.
Game 404 is converted.
Game 405 is converted.
Game 406 is converted.
Game 407 is converted.
Game 408 is converted.
Game 409 is converted.
Game 410 is

Game 724 is converted.
Game 725 is converted.
Game 726 is converted.
Game 727 is converted.
Game 728 is converted.
Game 729 is converted.
Game 730 is converted.
Game 731 is converted.
Game 732 is converted.
Game 733 is converted.
Game 734 is converted.
Game 735 is converted.
Game 736 is converted.
Game 737 is converted.
Game 738 is converted.
Game 739 is converted.
Game 740 is converted.
Game 741 is converted.
Game 742 is converted.
Game 743 is converted.
Game 744 is converted.
Game 745 is converted.
Game 746 is converted.
Game 747 is converted.
Game 748 is converted.
Game 749 is converted.
Game 750 is converted.
Game 751 is converted.
Game 752 is converted.
Game 753 is converted.
Game 754 is converted.
Game 755 is converted.
Game 756 is converted.
Game 757 is converted.
Game 758 is converted.
Game 759 is converted.
Game 760 is converted.
Game 761 is converted.
Game 762 is converted.
Game 763 is converted.
Game 764 is converted.
Game 765 is converted.
Game 766 is converted.
Game 767 is

Game 1078 is converted.
Game 1079 is converted.
Game 1080 is converted.
Game 1081 is converted.
Game 1082 is converted.
Game 1083 is converted.
Game 1084 is converted.
Game 1085 is converted.
Game 1086 is converted.
Game 1087 is converted.
Game 1088 is converted.
Game 1089 is converted.
Game 1090 is converted.
Game 1091 is converted.
Game 1092 is converted.
Game 1093 is converted.
Game 1094 is converted.
Game 1095 is converted.
Game 1096 is converted.
Game 1097 is converted.
Game 1098 is converted.
Game 1099 is converted.
Game 1100 is converted.
Game 1101 is converted.
Game 1102 is converted.
Game 1103 is converted.
Game 1104 is converted.
Game 1105 is converted.
Game 1106 is converted.
Game 1107 is converted.
Game 1108 is converted.
Game 1109 is converted.
Game 1110 is converted.
Game 1111 is converted.
Game 1112 is converted.
Game 1113 is converted.
Game 1114 is converted.
Game 1115 is converted.
Game 1116 is converted.
Game 1117 is converted.
Game 1118 is converted.
Game 1119 is con

Game 1442 is converted.
Game 1443 is converted.
Game 1444 is converted.
Game 1445 is converted.
Game 1446 is converted.
Game 1447 is converted.
Game 1448 is converted.
Game 1449 is converted.
Game 1450 is converted.
Game 1451 is converted.
Game 1452 is converted.
Game 1453 is converted.
Game 1454 is converted.
Game 1455 is converted.
Game 1456 is converted.
Game 1457 is converted.
Game 1458 is converted.
Game 1459 is converted.
Game 1460 is converted.
Game 1461 is converted.
Game 1462 is converted.
Game 1463 is converted.
Game 1464 is converted.
Game 1465 is converted.
Game 1466 is converted.
Game 1467 is converted.
Game 1468 is converted.
Game 1469 is converted.
Game 1470 is converted.
Game 1471 is converted.
Game 1472 is converted.
Game 1473 is converted.
Game 1474 is converted.
Game 1475 is converted.
Game 1476 is converted.
Game 1477 is converted.
Game 1478 is converted.
Game 1479 is converted.
Game 1480 is converted.
Game 1481 is converted.
Game 1482 is converted.
Game 1483 is con

Game 1804 is converted.
Game 1805 is converted.
Game 1806 is converted.
Game 1807 is converted.
Game 1808 is converted.
Game 1809 is converted.
Game 1810 is converted.
Game 1811 is converted.
Game 1812 is converted.
Game 1813 is converted.
Game 1814 is converted.
Game 1815 is converted.
Game 1816 is converted.
Game 1817 is converted.
Game 1818 is converted.
Game 1819 is converted.
Game 1820 is converted.
Game 1821 is converted.
Game 1822 is converted.
Game 1823 is converted.
Game 1824 is converted.
Game 1825 is converted.
Game 1826 is converted.
Game 1827 is converted.
Game 1828 is converted.
Game 1829 is converted.
Game 1830 is converted.
Game 1831 is converted.
Game 1832 is converted.
Game 1833 is converted.
Game 1834 is converted.
Game 1835 is converted.
Game 1836 is converted.
Game 1837 is converted.
Game 1838 is converted.
Game 1839 is converted.
Game 1840 is converted.
Game 1841 is converted.
Game 1842 is converted.
Game 1843 is converted.
Game 1844 is converted.
Game 1845 is con

Game 2159 is converted.
Game 2160 is converted.
Game 2161 is converted.
Game 2162 is converted.
Game 2163 is converted.
Game 2164 is converted.
Game 2165 is converted.
Game 2166 is converted.
Game 2167 is converted.
Game 2168 is converted.
Game 2169 is converted.
Game 2170 is converted.
Game 2171 is converted.
Game 2172 is converted.
Game 2173 is converted.
Game 2174 is converted.
Game 2175 is converted.
Game 2176 is converted.
Game 2177 is converted.
Game 2178 is converted.
Game 2179 is converted.
Game 2180 is converted.
Game 2181 is converted.
Game 2182 is converted.
Game 2183 is converted.
Game 2184 is converted.
Game 2185 is converted.
Game 2186 is converted.
Game 2187 is converted.
Game 2188 is converted.
Game 2189 is converted.
Game 2190 is converted.
Game 2191 is converted.
Game 2192 is converted.
Game 2193 is converted.
Game 2194 is converted.
Game 2195 is converted.
Game 2196 is converted.
Game 2197 is converted.
Game 2198 is converted.
Game 2199 is converted.
Game 2200 is con

Game 2508 is converted.
Game 2509 is converted.
Game 2510 is converted.
Game 2511 is converted.
Game 2512 is converted.
Game 2513 is converted.
Game 2514 is converted.
Game 2515 is converted.
Game 2516 is converted.
Game 2517 is converted.
Game 2518 is converted.
Game 2519 is converted.
Game 2520 is converted.
Game 2521 is converted.
Game 2522 is converted.
Game 2523 is converted.
Game 2524 is converted.
Game 2525 is converted.
Game 2526 is converted.
Game 2527 is converted.
Game 2528 is converted.
Game 2529 is converted.
Game 2530 is converted.
Game 2531 is converted.
Game 2532 is converted.
Game 2533 is converted.
Game 2534 is converted.
Game 2535 is converted.
Game 2536 is converted.
Game 2537 is converted.
Game 2538 is converted.
Game 2539 is converted.
Game 2540 is converted.
Game 2541 is converted.
Game 2542 is converted.
Game 2543 is converted.
Game 2544 is converted.
Game 2545 is converted.
Game 2546 is converted.
Game 2547 is converted.
Game 2548 is converted.
Game 2549 is con

Game 2881 is converted.
Game 2882 is converted.
Game 2883 is converted.
Game 2884 is converted.
Game 2885 is converted.
Game 2886 is converted.
Game 2887 is converted.
Game 2888 is converted.
Game 2889 is converted.
Game 2890 is converted.
Game 2891 is converted.
Game 2892 is converted.
Game 2893 is converted.
Game 2894 is converted.
Game 2895 is converted.
Game 2896 is converted.
Game 2897 is converted.
Game 2898 is converted.
Game 2899 is converted.
Game 2900 is converted.
Game 2901 is converted.
Game 2902 is converted.
Game 2903 is converted.
Game 2904 is converted.
Game 2905 is converted.
Game 2906 is converted.
Game 2907 is converted.
Game 2908 is converted.
Game 2909 is converted.
Game 2910 is converted.
Game 2911 is converted.
Game 2912 is converted.
Game 2913 is converted.
Game 2914 is converted.
Game 2915 is converted.
Game 2916 is converted.
Game 2917 is converted.
Game 2918 is converted.
Game 2919 is converted.
Game 2920 is converted.
Game 2921 is converted.
Game 2922 is con

Game 3233 is converted.
Game 3234 is converted.
Game 3235 is converted.
Game 3236 is converted.
Game 3237 is converted.
Game 3238 is converted.
Game 3239 is converted.
Game 3240 is converted.
Game 3241 is converted.
Game 3242 is converted.
Game 3243 is converted.
Game 3244 is converted.
Game 3245 is converted.
Game 3246 is converted.
Game 3247 is converted.
Game 3248 is converted.
Game 3249 is converted.
Game 3250 is converted.
Game 3251 is converted.
Game 3252 is converted.
Game 3253 is converted.
Game 3254 is converted.
Game 3255 is converted.
Game 3256 is converted.
Game 3257 is converted.
Game 3258 is converted.
Game 3259 is converted.
Game 3260 is converted.
Game 3261 is converted.
Game 3262 is converted.
Game 3263 is converted.
Game 3264 is converted.
Game 3265 is converted.
Game 3266 is converted.
Game 3267 is converted.
Game 3268 is converted.
Game 3269 is converted.
Game 3270 is converted.
Game 3271 is converted.
Game 3272 is converted.
Game 3273 is converted.
Game 3274 is con

In [10]:
not_conv_games

[]

In [12]:
X = np.array(input_data)
y = np.array(output_data).reshape(len(output_data), 4)

In [13]:
X[0], y[0]

(array([12,  9,  7, 11,  8,  7,  9, 12, 10, 10, 10, 10, 10, 10, 10, 10,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  4,  4,
         4,  4,  4,  4,  4,  6,  3,  1,  5,  2,  1,  3,  6]),
 array([6, 4, 4, 4]))

In [14]:
y_ = []

for o in y:
    y_.append([bi for out in o for bi in binary_3bits(out)])

In [15]:
y = np.array(y_)

In [16]:
X.shape, y.shape

((157787, 64), (157787, 12))

In [17]:
X.reshape(len(X), 8, 8)[3], y[3]

(array([[12,  0,  7, 11,  8,  7,  9, 12],
        [10, 10,  0,  0, 10, 10, 10, 10],
        [ 0,  0,  9, 10,  0,  0,  0,  0],
        [ 0,  0, 10,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  4,  4,  0,  0],
        [ 0,  0,  3,  0,  0,  0,  0,  0],
        [ 4,  4,  4,  4,  0,  0,  4,  4],
        [ 6,  0,  1,  5,  2,  1,  3,  6]]),
 array([1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1]))

In [ ]:
with open("../dataset/levon_input.pickle", "wb") as file:
    pickle.dump(X, file)
    
with open("../dataset/levon_output.pickle", "wb") as file:
    pickle.dump(y, file)